In [1]:
import random

random.seed(123)

## IC

In [2]:
import numpy as np
import pandas
np.random.seed(123)
# Parameters
m = 100
sim = 10000
n = 5
exp_w = (n * (n + m + 1)) / 2
stdev_w = np.sqrt(n * m * (n + m + 1) / 12)
lambda_ = 0.5
Le = 2.188
L_SSGR =  3

# Control Limits
LCL = exp_w - Le * stdev_w * np.sqrt(lambda_ / (2 - lambda_))
UCL = exp_w + Le * stdev_w * np.sqrt(lambda_ / (2 - lambda_))

# Simulating Phase I and Phase II
run_lengths = []
total_runs = []
for i in range(sim):
    zi_1 = exp_w  # Phase I mean as starting EWMA value
    xi = np.random.normal(0, 1, m)  # Phase I data
    count = 0
    signaled = False
    total_run = 0
    while not signaled:
        count += 1
        total_run += 1
        yi = np.random.normal(0, 1, n)  # Phase II data
        comb = np.concatenate((xi, yi))
        ranks = np.argsort(np.argsort(comb)) + 1  # Rank calculation
        W = ranks[-n:].sum()  # Sum of ranks for new sample

        # Calculate W-EWMA
        zi = lambda_ * W + (1 - lambda_) * zi_1
        zi_1 = zi
        # Check control limits and evaluate run length
        if zi >= UCL or zi <= LCL:
            if count <= L_SSGR:
                run_lengths.append(count)
                total_runs.append(total_run)
                signaled = True


            count = 0  # Reset count after nonconforming group and no signal

run_lengths = np.array(total_runs)
print(f"Mean Run Length: {run_lengths.mean()}")
print(f"Standard Deviation: {run_lengths.std()}")
print(f"5th Percentile: {np.percentile(run_lengths, 5)}")
print(f"95th Percentile: {np.percentile(run_lengths, 95)}")
print(f"Median: {np.percentile(run_lengths, 50)}")

Mean Run Length: 202.6426
Standard Deviation: 267.52315612903493
5th Percentile: 3.0
95th Percentile: 696.0
Median: 111.0


## OOC

In [5]:
import numpy as np
import pandas as pd
from tqdm import tqdm

# Parameters
m = 100
sim = 10000
n = 5
exp_w = (n * (n + m + 1)) / 2
stdev_w = np.sqrt(n * m * (n + m + 1) / 12)
lambda_ = 0.5
# Le = 2.345    # For 0.05
Le = 2.188      # For 0.5
L_SSGR =  3
df = 5
# df = 20
# shape_param = 1  # Shape parameter for the gamma distribution
shape_param = 15  # Shape parameter for the gamma distribution
scale_param = 1  # Scale parameter for the gamma distribution
# Control Limits
LCL = exp_w - Le * stdev_w * np.sqrt(lambda_ / (2 - lambda_))
UCL = exp_w + Le * stdev_w * np.sqrt(lambda_ / (2 - lambda_))

deltas = np.arange(0, 3.25, 0.25)
results = pd.DataFrame(columns=['Delta', 'ARL', 'SDRL', 'MDRL'])

# Simulating Phase I and Phase II


for delta in deltas:
    print(delta)
    run_lengths = []
    total_runs = []
    for i in tqdm(range(sim)):
        zi_1 = exp_w  # Phase I mean as starting EWMA value
        xi = np.random.normal(0, 1, m)  # Phase I data Normal
        # xi = np.random.standard_t(df, m)  # Phase I data t
        # xi = np.random.gamma(shape_param, scale_param, m)  # Phase I data Gamma
        # xi=(xi-shape_param)/np.sqrt(shape_param)
        count = 0
        signaled = False
        total_run = 0
        while not signaled:
            count += 1
            total_run += 1
            yi = np.random.normal(delta, 1, n)  # Phase II data
            # yi = np.random.standard_t(df, n) + delta  # Phase II data t
            # y1 = np.random.gamma(shape_param, scale_param, n)  # Phase II data Gamma
            # yi = (y1 - shape_param) / np.sqrt(shape_param) + (delta)
            comb = np.concatenate((xi, yi))
            ranks = np.argsort(np.argsort(comb)) + 1  # Rank calculation
            W = ranks[-n:].sum()  # Sum of ranks for new sample

            # Calculate W-EWMA
            zi = lambda_ * W + (1 - lambda_) * zi_1
            zi_1 = zi
            # Check control limits and evaluate run length
            if zi >= UCL or zi <= LCL:
                if count <= L_SSGR:
                    # run_lengths.append(count)
                    total_runs.append(total_run)
                    signaled = True


                count = 0  # Reset count after nonconforming group and no signal

    run_lengths = np.array(total_runs)
    ARL = run_lengths.mean()
    SDRL = run_lengths.std()
    MDRL = np.percentile(run_lengths, 50)

    # Append to results
    results_row = pd.DataFrame({'Delta': [delta], 'ARL': [ARL], 'SDRL': [SDRL], 'MDRL': [MDRL]})
    results = pd.concat([results, results_row], ignore_index=True)
    
results


0.0


100%|██████████| 10000/10000 [00:25<00:00, 390.62it/s]
C:\Users\hilar\AppData\Local\Temp\ipykernel_10736\1614036597.py:74: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, results_row], ignore_index=True)


0.25


100%|██████████| 10000/10000 [00:07<00:00, 1284.72it/s]


0.5


...

2.75


100%|██████████| 10000/10000 [00:00<00:00, 59320.42it/s]


3.0


100%|██████████| 10000/10000 [00:00<00:00, 52322.98it/s]


,Delta,ARL,SDRL,MDRL
0,0.00,203.2899,277.389688,112.0
1,0.25,61.2098,108.699023,26.0
2,0.50,10.1097,12.988644,6.0
3,0.75,3.7859,3.271156,3.0
4,1.00,2.2766,1.381771,2.0
5,1.25,1.6916,0.813935,2.0
6,1.50,1.3864,0.557221,1.0
7,1.75,1.2058,0.413578,1.0
8,2.00,1.0975,0.298653,1.0
9,2.25,1.0362,0.187322,1.0


# Multiple Comparison

In [4]:
import numpy as np
import numpy
import pandas as pd
from tqdm import tqdm
import pandas

# Parameters
m = 100
sim = 10000
n = 5
exp_w = (n * (n + m + 1)) / 2
stdev_w = np.sqrt(n * m * (n + m + 1) / 12)
lambda_ = 0.5
# Le = 2.345    # For 0.05
# Le = 2.188      # For 0.5
L_SSGR =  3
df = 5
# df = 20
shape_param = 1  # Shape parameter for the gamma distribution
# shape_param = 15  # Shape parameter for the gamma distribution
scale_param = 1  # Scale parameter for the gamma distribution

lambda_values = [0.05, 0.5]
Le_values = [2.345, 2.188]

deltas = np.arange(0, 3.25, 0.25)

results = pandas.DataFrame(columns=['Delta', 'ARL (0.05)', 'SDRL (0.05)', 'MDRL (0.05)', 'ARL (0.5)', 'SDRL (0.5)', 'MDRL (0.5)'])

for lambda_, Le in zip(lambda_values, Le_values):
    LCL = exp_w - Le * stdev_w * numpy.sqrt(lambda_ / (2 - lambda_))
    UCL = exp_w + Le * stdev_w * numpy.sqrt(lambda_ / (2 - lambda_))
    arl_column = f'ARL ({lambda_})'
    sdrl_column = f'SDRL ({lambda_})'
    mdrl_column = f'MDRL ({lambda_})'

    for delta in deltas:
        print(delta)
        run_lengths = []
        total_runs = []
        for i in tqdm(range(sim)):
            zi_1 = exp_w  # Phase I mean as starting EWMA value
            # xi = np.random.normal(0, 1, m)  # Phase I data Normal
            # xi = np.random.standard_t(df, m)  # Phase I data t
            xi = np.random.gamma(shape_param, scale_param, m)  # Phase I data Gamma
            xi=(xi-shape_param)/np.sqrt(shape_param)
            count = 0
            signaled = False
            total_run = 0
            while not signaled:
                count += 1
                total_run += 1
                # yi = np.random.normal(delta, 1, n)  # Phase II data
                # yi = np.random.standard_t(df, n) + delta  # Phase II data t
                y1 = np.random.gamma(shape_param, scale_param, n)  # Phase II data Gamma
                yi = (y1 - shape_param) / np.sqrt(shape_param) + (delta)
                comb = np.concatenate((xi, yi))
                ranks = np.argsort(np.argsort(comb)) + 1  # Rank calculation
                W = ranks[-n:].sum()  # Sum of ranks for new sample

                # Calculate W-EWMA
                zi = lambda_ * W + (1 - lambda_) * zi_1
                zi_1 = zi
                # Check control limits and evaluate run length
                if zi >= UCL or zi <= LCL:
                    if count <= L_SSGR:
                        # run_lengths.append(count)
                        total_runs.append(total_run)
                        signaled = True


                    count = 0  # Reset count after nonconforming group and no signal

        run_lengths = np.array(total_runs)
        ARL = run_lengths.mean()
        SDRL = run_lengths.std()
        MDRL = np.percentile(run_lengths, 50)

        if delta in results['Delta'].values:
            results.loc[results['Delta'] == delta, arl_column] = ARL
            results.loc[results['Delta'] == delta, sdrl_column] = SDRL
            results.loc[results['Delta'] == delta, mdrl_column] = MDRL
        else:
            new_row = pandas.DataFrame({'Delta': [delta], arl_column: [ARL], sdrl_column: [SDRL], mdrl_column: [MDRL]})
            results = pandas.concat([results, new_row], ignore_index=True)

results['ARL'] = results.apply(lambda row: f"({row['ARL (0.05)']:.2f}, {row['ARL (0.5)']:.2f})", axis=1)
results = results[['Delta', 'ARL']]

results


0.0


100%|██████████| 10000/10000 [01:12<00:00, 137.46it/s]


0.25


100%|██████████| 10000/10000 [00:06<00:00, 1546.17it/s]


0.5


...

2.75


100%|██████████| 10000/10000 [00:00<00:00, 20286.98it/s]


3.0


100%|██████████| 10000/10000 [00:00<00:00, 18837.43it/s]


,Delta,ARL
0,0.00,"(208.28, 202.29)"
1,0.25,"(19.11, 42.94)"
2,0.50,"(7.57, 5.11)"
3,0.75,"(5.66, 2.27)"
4,1.00,"(4.70, 1.66)"
5,1.25,"(3.81, 1.31)"
6,1.50,"(3.21, 1.09)"
7,1.75,"(3.03, 1.02)"
8,2.00,"(3.00, 1.00)"
9,2.25,"(3.00, 1.00)"
